In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from evaluate.evaluator import Evaluator
from visualize import SearchPlotter


from algs.mealpy_algorithm import MealpyAlgorithm
from mealpy.physics_based.CDO import OriginalCDO

plotter = SearchPlotter()

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

loss_func = loss_funcs.StructuralSimilarity()

alg = MealpyAlgorithm(sim_viewer, loss_func=loss_func, optimizer=OriginalCDO())
alg_config = MealpyAlgorithm.Config(time_limit=10, seed=1, run_mode="single")

evaluator = Evaluator(world_viewer, eval_func=eval_funcs.NormXorDiff(obj_depth=1.0, method="mae"))

init_location = (0, 1.3, 0.3)
random_orientations = np.random.uniform(0, 2 * np.pi, size=(5, 3))
eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

for alg, config in [(alg, alg_config)]:
    eval_losses = evaluator.evaluate(alg, config, eval_positions)
    print(f"{type(alg).__name__}: {eval_losses}")

plotter.close()
world_viewer.close()